In [2]:
#the Fills that can be used

#2017
#better BCMS BSRT after 6050->to use
#flist_BCMS_afterCalibr = [6054, 6055, 6057, 6060, 6061, 6072, 6079, 6082, 6084, 6086, 6089, 6090, 6091, 6093, 6094, 6096, 6104, 6105, 6106, 6110, 6114, 6116, 6119, 6123, 6136, 6138, 6140, 6141, 6142, 6143, 6146, 6147, 6152, 6155, 6156, 6158, 6159, 6160, 6161, 6165]
#better 8b4e BSRT after 6200->to use
#flist_8b4e = [6167, 6168, 6169, 6170, 6171, 6174, 6175, 6176, 6177, 6179, 6180, 6182, 6185, 6186, 6189, 6192, 6193, 6238, 6239, 6240, 6241, 6243, 6245, 6247, 6252, 6253, 6255, 6258, 6259, 6261, 6262, 6263] 
#better 8b4eBCS BSRT after 6310->to use
#flist_8b4eBCS= [6266, 6268, 6269,  6278, 6279, 6283, 6285, 6287, 6288, 6291, 6297, 6298, 6300, 6303, 6304, 6305, 6306, 6307, 6308, 6309, 6311, 6312, 6313, 6314, 6315, 6317, 6318, 6323, 6324, 6325, 6336, 6337, 6341, 6343, 6344, 6346, 6347, 6348, 6349, 6351, 6355, 6356, 6358, 6360, 6362, 6364, 6370, 6371] 

#2018, some trusted BSRT fills...->to use
#flist_BCMS=[6646, 6741, 6747, 6751, 6752, 6755, 6763, 6768, 6770, 6772, 6773, 6776, 6929, 6942,6946, 6953, 6957, 6960, 6961, 7005, 7006, 7013, 7018, 7020, 7031, 7035, 7037, 7039, 7040, 7069, 7078, 7080, 7090, 7091, 7092, 7105, 7221, 7234, 7309, 7317, 7320, 7321,7324, 7331, 7333, 7334]

In [1]:
#%run -i ~/myLibrary/init.ipynb
from scipy.special   import erfc
from scipy.integrate import cumtrapz
from math import *
from scipy.constants import c
import matplotlib.gridspec as gridspec
from scipy.optimize import fsolve
import numpy as np
import matplotlib.pyplot as pl
import pickle, gzip
na = np.array
from scipy.constants import c as clight

## Parameters for coupling, burn-off and noise

In [2]:

dt_step=5.0           #for a dt_step=5 min step since the data are saved every 5min

##parameters for coupling
Coupling_coeff_FT            = 0.0015    # Linear coupling coefficient for Model in FT
Delta_tunesplit_FT           = 0.01      # Unperturbed tune split in FT 

##parameters for burn off
multip_factor=0.075/0.20 #around 20% of protons are lost because of Boff for an emit. growth of 7.5% by the end of the fill, based on slide 25 in https://indico.cern.ch/event/806869/contributions/3358585/attachments/1816649/2969341/Emittance_BU_due_to_BO.pdf

##parameters for noise
theGainFT=0.025
gains = np.arange(0.0,0.2,0.001)
#noise floor of the machine
delta_hb1 = 3.8e-5
delta_vb1 = 5.3e-5
delta_hb2 = 4.4e-5
delta_vb2 = 5.6e-5
#noise floor of the transverse feedback ADT 
deltaBPM_SB_hb1 = 220e-5
deltaBPM_SB_vb1 = 250e-5
deltaBPM_SB_hb2 = 190e-5
deltaBPM_SB_vb2 = 210e-5

r0 = 1.535E-18
fr = 11.2455e3
frev=fr

#deltaBPMnow=deltaBPM*(1.2e11/Inow)^k, deltaBPM varies with intensity, for k=1/2?
k=1./2. #this value would be more or less the same for both planes and beams
#dmu for non-colliding, dmu_nonHILUMI = 9.3e-5 #varies linearly with 2.5emit, 570A  and 1/7TeV, so.... https://cds.cern.ch/record/2301292/files/CERN-ACC-NOTE-2018-0002.pdf
dmu_non_factor=9.3e-5/2.5e-6

#call elastic_scattering, IBSmodel, noise functions and LumiModel with coupling, noise, burn off
P = 6500. #for 6.5TeV in elastic scattering function
%run -i emit_model_elastic_scattering.ipynb
%run -i IBSmodel_GY.ipynb
%run -i noiseINmodel_new.ipynb
%run -i LumiModel_CouplNoiseBoff_new.ipynb

In [3]:
##beam parameters
gamma = 6927.64 #6.5TeV
VRF_V = 12e6
tauSRxy_s = 64.7*3600;      # energy dependent
tauSRl_s = 32.35*3600;      # energy dependent
sigmaBOff_m2 = 80*1e-31;    # energy dependent
sigma_el_m2 = 29.7*1e-31;   # energy dependent

In [6]:
#print "HB1","","VB1","","HB2","","VB2"
#corEmitvalue=1.9
#print corEmitvalue/np.mean(dict_fill_SB['eh_interp_coll'][1][0]),corEmitvalue/np.mean(dict_fill_SB['ev_interp_coll'][1][0]), corEmitvalue/np.mean(dict_fill_SB['eh_interp_coll'][2][0]),corEmitvalue/np.mean(dict_fill_SB['ev_interp_coll'][2][0])

In [8]:
#correct BSRT by multiplying the emittances by a cor. factor, if not then set cor_fact_*=1.
#to find mult. factors we should see what is the emittance inferred by luminosity and meit. scans.
#then aslo lumi model should agree better with the measured lumi

dict_case = {}
dict_case['case'] = [2]


if dict_case['case'] == [1]: #uncorrected
    dict_case['cor_fact_1h'] = [1.]     # 100 = uncorrected , 1.1 = 110% = +10% corr.
    dict_case['cor_fact_1v'] = [1.]
    dict_case['cor_fact_2h'] = [1.]
    dict_case['cor_fact_2v'] = [1.]
    print "case1: uncorrected BSRT"

elif dict_case['case'] == [2]: #corrected
    #use the factors you want
    
    c1h = 1.09;c1v = 1.1;c2h = 1.25;c2v = 1.05;    #FILL 7035
    #c1h = 1.;c1v = 1.22;c2h = 1.22;c2v = 1.08; #FILL 7221
    #c1h = 1.1;c1v = 1.;c2h = 1.1;c2v = 1.1;    #FILL 7334
    
    dict_case['cor_fact_1h'] = [c1h]     # 100 = uncorrected , 1.1 = 110% = +10% corr.
    dict_case['cor_fact_1v'] = [c1v]
    dict_case['cor_fact_2h'] = [c2h]
    dict_case['cor_fact_2v'] = [c2v]
    print "case2: correct BSRT by multiplying the emittances by a cor. factor"
    

cor_fact_1h = dict_case['cor_fact_1h'][0]
cor_fact_1v = dict_case['cor_fact_1v'][0]
cor_fact_2h = dict_case['cor_fact_2h'][0]
cor_fact_2v = dict_case['cor_fact_2v'][0]
    
print "HB1","","VB1","","HB2","","VB2"
print  cor_fact_1h,cor_fact_1v,cor_fact_2h,cor_fact_2v

case2: correct BSRT by multiplying the emittances by a cor. factor
HB1  VB1  HB2  VB2
1.09 1.1 1.25 1.05


In [11]:
#define the fill list you want to run

#fill_list=[6104, 6105, 6106, 6110, 6114, 6116, 6119, 6123, 6136, 6138, 6140, 6141, 6142, 6143, 6146, 6147, 6152, 6155, 6156, 6158, 6159, 6160, 6161, 6165]
fill_list=[6238, 6239, 6241, 6243, 6245, 6247, 6252, 6253, 6255, 6258, 6259, 6261, 6262, 6263] 
#fill_list=[6360, 6362, 6364, 6371]
#better 8b4eBCS BSRT after 6310->to use

for filln in fill_list:
    
    noAntiLeveling=0 #if 1 keep both xing and beta* constant

    #2018
    if filln > 6500:  
        print 'fill%d,'%filln, '2018'
        theyear=2018
        if filln==7006:
            print "this fill has one beam that is missing a train (non-colliding)"    
    #2017        
    elif   filln >= 5246 and filln <= 6500: 
        theyear=2017
        if filln >= 6054 and filln <= 6165: 
            print 'fill%d,'%filln, '2017 BCMS'
        if filln >= 6167 and filln <= 6263:
            print 'fill%d,'%filln, '2017 8b4e'
        if filln >= 6266 and filln <= 6371:    
            print 'fill%d,'%filln, '2017 8b4eBCS'
        
    with gzip.open('/eos/project/l/lhc-lumimod/LuminosityFollowUp/%d/procdata/fill_%d/fill_%d_fits.pkl.gz'%(theyear,filln, filln), 'rb') as fid:
        dict_fill_fits = pickle.load(fid)
    with gzip.open('/eos/project/l/lhc-lumimod/LuminosityFollowUp/%d/procdata/fill_%d/fill_%d.pkl.gz'%(theyear,filln, filln), 'rb') as fid:
        dict_fill_SB = pickle.load(fid)
    with gzip.open('/eos/project/l/lhc-lumimod/LuminosityFollowUp/%d/procdata/fill_%d/fill_%d_lumi_meas.pkl.gz'%(theyear,filln, filln), 'rb') as fid:
        dict_lumis = pickle.load(fid)
        
    if filln > 6400:
        betastar_m = dict_fill_SB['betastar'][1]/100     
    elif filln > 5800 and filln < 6194:
        betastar_m = 0.4
    elif filln > 6194 and filln < 6400:    
        betastar_m = 0.3

    phi_full_rad_ATLAS = dict_fill_SB['xing_angle'][1]
    phi_full_rad_CMS = dict_fill_SB['xing_angle'][5]

    if noAntiLeveling:
        phi_full_rad_ATLAS=[phi_full_rad_ATLAS[0]]*len(phi_full_rad_ATLAS)
        phi_full_rad_CMS=[phi_full_rad_CMS[0]]*len(phi_full_rad_CMS)
       
    print 'xingAngle:',phi_full_rad_ATLAS#, 'and',phi_full_rad_CMS
    print 'beta*:',betastar_m     



    dict_models_settings={}
    dict_models_settings['IBSBOff'] = {
'tau_empirical_h1_coll':None,
'tau_empirical_v1_coll':None,
    'tau_empirical_h2_coll':None,
'tau_empirical_v2_coll':None,

'tau_empirical_h1_noncoll':None,
'tau_empirical_v1_noncoll':None,
'tau_empirical_h2_noncoll':None,
'tau_empirical_v2_noncoll':None,
'BOff': 1,
'blengthBU':"Model",
'emitBU':"Model"}

    dict_models_settings['IBSLosses'] = {
'tau_empirical_h1_coll':None,
'tau_empirical_v1_coll':None,
'tau_empirical_h2_coll':None,
'tau_empirical_v2_coll':None,

'tau_empirical_h1_noncoll':None,
'tau_empirical_v1_noncoll':None,
'tau_empirical_h2_noncoll':None,
'tau_empirical_v2_noncoll':None,
'BOff': 0,
'blengthBU':"Model",
'emitBU':"Model"}


    dict_models_settings['EmpiricalBlowupBOff'] = {
'tau_empirical_h1_coll':dict_fill_fits['beam_1']['tau_emith_coll_full'],
'tau_empirical_v1_coll':dict_fill_fits['beam_1']['tau_emitv_coll_full'],
'tau_empirical_h2_coll':dict_fill_fits['beam_2']['tau_emith_coll_full'],
'tau_empirical_v2_coll':dict_fill_fits['beam_2']['tau_emitv_coll_full'],

'tau_empirical_h1_noncoll':dict_fill_fits['beam_1']['tau_emith_noncoll_full'],
'tau_empirical_v1_noncoll':dict_fill_fits['beam_1']['tau_emitv_noncoll_full'],
'tau_empirical_h2_noncoll':dict_fill_fits['beam_2']['tau_emith_noncoll_full'],
'tau_empirical_v2_noncoll':dict_fill_fits['beam_2']['tau_emitv_noncoll_full'],
'BOff': 1,
'blengthBU':"Model",
'emitBU':"EmpiricalBlowup"}

    dict_models_settings['EmpiricalBlowupLosses'] = {
'tau_empirical_h1_coll':dict_fill_fits['beam_1']['tau_emith_coll_full'],
'tau_empirical_v1_coll':dict_fill_fits['beam_1']['tau_emitv_coll_full'],
'tau_empirical_h2_coll':dict_fill_fits['beam_2']['tau_emith_coll_full'],
'tau_empirical_v2_coll':dict_fill_fits['beam_2']['tau_emitv_coll_full'],

'tau_empirical_h1_noncoll':dict_fill_fits['beam_1']['tau_emith_noncoll_full'],
'tau_empirical_v1_noncoll':dict_fill_fits['beam_1']['tau_emitv_noncoll_full'],
'tau_empirical_h2_noncoll':dict_fill_fits['beam_2']['tau_emith_noncoll_full'],
'tau_empirical_v2_noncoll':dict_fill_fits['beam_2']['tau_emitv_noncoll_full'],
'BOff': 0,
'blengthBU':"Model",
'emitBU':"EmpiricalBlowup"}

    dict_models = {}

    if np.logical_and(filln > 6274,filln<6380):
        leveling = 1.5e38
    else:
        leveling = 0
        #for model_name in ['pure', 'EmpiricalBlowup', 'EmpiricalBlowupFull']:

    
    #[coupling,noiseGrowth,boffGrowth]
    modelCase0=[0,0,0]  #"old model"
    modelCase1=[1,0,0]  #"coupling"
    modelCase2=[1,1,0]  #"coupling&noise"
    modelCase3=[1,0,1]  #"coupling&boff"
    modelCase4=[1,1,1]  #"coupling&noise&boff"

    for modelCase in [modelCase0,modelCase1,modelCase2,modelCase3,modelCase4]:
        #print '---------------------model case:',modelCase
        [coupling,noiseGrowth,boffGrowth]=modelCase
        #print coupling,noiseGrowth,boffGrowth        

        if modelCase==modelCase0:
                    print 'Running  "old model" for fill%d'%filln
        if modelCase==modelCase1:
                    print 'Running  "coupling"  for fill%d'%filln
        if modelCase==modelCase2:       
                    print 'Running  "coupling&noise"  for fill%d'%filln
        if modelCase==modelCase3:       
                    print 'Running  "coupling&boff"  for fill%d'%filln            
        if modelCase==modelCase4:
                    print 'Running  "coupling&noise&boff"  for fill%d'%filln
                    
        
        #run only for the model cases where we take emittance from model, since coupling flag is to be used only on the model, not on the data emittances
        for model_name in ['IBSLosses','IBSBOff']:#dict_models_settings.keys():

            this_model = dict_models_settings[model_name]
        #print model_name
            #handle colliding
            family = 'coll'
            #bunch_intensity_b1_init =  dict_fill_SB['b_inten_interp_%s'%family][1][0,:]
            #bunch_intensity_b2_init =  dict_fill_SB['b_inten_interp_%s'%family][2][0,:]
            bunch_intensity_b1_init =  dict_fill_SB['b_inten_interp_%s'%family][1]
            bunch_intensity_b2_init =  dict_fill_SB['b_inten_interp_%s'%family][2]

            #emith_b1_init =  dict_fill_SB['eh_interp_%s'%family][1][0,:]*cor_fact_1h
            #emith_b2_init =  dict_fill_SB['eh_interp_%s'%family][2][0,:]*cor_fact_2h

            #emitv_b1_init =  dict_fill_SB['ev_interp_%s'%family][1][0,:]*cor_fact_1v
            #emitv_b2_init =  dict_fill_SB['ev_interp_%s'%family][2][0,:]*cor_fact_2v

            emith_b1_init =  dict_fill_SB['eh_interp_%s'%family][1]*cor_fact_1h
            emith_b2_init =  dict_fill_SB['eh_interp_%s'%family][2]*cor_fact_2h

            emitv_b1_init =  dict_fill_SB['ev_interp_%s'%family][1]*cor_fact_1v
            emitv_b2_init =  dict_fill_SB['ev_interp_%s'%family][2]*cor_fact_2v
            #print emith_b1_init

            blen_b1_init =  dict_fill_SB['bl_interp_m_%s'%family][1]
            blen_b2_init =  dict_fill_SB['bl_interp_m_%s'%family][2]


            trange = dict_fill_SB['time_range']
            tFill_s  = (trange[-1]-trange[0])

            (tt_s, Luminosity_invm2s_model_ATLAS, Luminosity_invm2s_model_CMS, bunch_intensity_p1_mod, bunch_intensity_p2_mod,
		ex_norm_m1_mod, ex_norm_m2_mod, ey_norm_m1_mod, ey_norm_m2_mod, bl_4sigma_s1_mod,
		bl_4sigma_s2_mod, ex_norm_m1_mod_IBScorr, ex_norm_m2_mod_IBScorr, ey_norm_m1_mod_IBScorr, ey_norm_m2_mod_IBScorr, 
		dex_noise1,dey_noise1,dex_noise2,dey_noise2, deltaBPMnow_hb1,deltaBPMnow_vb1,deltaBPMnow_hb2,deltaBPMnow_vb2, xi_hb1,xi_vb1,xi_hb2,xi_vb2, de_boff1h,de_boff1v,de_boff2h,de_boff2v) =\
            LumiModel(gamma=gamma, betastar_m=betastar_m, phi_full_rad_ATLAS=phi_full_rad_ATLAS,
		phi_full_rad_CMS=phi_full_rad_CMS,
		bunch_intensityin_p1=bunch_intensity_b1_init, bunch_intensityin_p2=bunch_intensity_b2_init,
		exin_norm_m1=emith_b1_init*1.0e-6, exin_norm_m2=emith_b2_init*1.0e-6,
		eyin_norm_m1=emitv_b1_init*1.0e-6, eyin_norm_m2=emitv_b2_init*1.0e-6,
		blin_4sigma_s1=blen_b1_init*4.0/clight,
		blin_4sigma_s2=blen_b2_init*4.0/clight,
		tFill_s=tFill_s,tauSRxy_s=tauSRxy_s, tauSRl_s=tauSRl_s, sigmaBOff_m2=sigmaBOff_m2, sigmaElastic_m2=sigma_el_m2, VRF_V=VRF_V,
		IBSON=1, emitBU=this_model['emitBU'], BoffON=this_model['BOff'], 
		coupling=coupling, noiseGrowth=noiseGrowth, boffGrowth=boffGrowth,          
		blengthBU=this_model['blengthBU'], nIPs = 2., dt_s=trange[1]-trange[0],
		tau_empirical_h1=this_model['tau_empirical_h1_coll'],
		tau_empirical_v1=this_model['tau_empirical_v1_coll'],
		tau_empirical_h2=this_model['tau_empirical_h2_coll'],
		tau_empirical_v2=this_model['tau_empirical_v2_coll'],
		leveling=leveling)

            eh_interp_coll = {1:[], 2:[]}
            ev_interp_coll = {1:[], 2:[]}

            eh_interp_coll_IBScorr = {1:[], 2:[]}
            ev_interp_coll_IBScorr = {1:[], 2:[]}
        
            b_inten_interp_coll = {1:[], 2:[]}
            bl_interp_m_coll = {1:[], 2:[]}


            eh_interp_coll[1] = na(ex_norm_m1_mod)*1.0e6
            eh_interp_coll[2] = na(ex_norm_m2_mod)*1.0e6

            ev_interp_coll[1] = na(ey_norm_m1_mod)*1.0e6
            ev_interp_coll[2] = na(ey_norm_m2_mod)*1.0e6

            eh_interp_coll_IBScorr[1] = na(ex_norm_m1_mod_IBScorr)*1.0e6
            eh_interp_coll_IBScorr[2] = na(ex_norm_m2_mod_IBScorr)*1.0e6

            ev_interp_coll_IBScorr[1] = na(ey_norm_m1_mod_IBScorr)*1.0e6
            ev_interp_coll_IBScorr[2] = na(ey_norm_m2_mod_IBScorr)*1.0e6

            b_inten_interp_coll[1] = na(bunch_intensity_p1_mod)
            b_inten_interp_coll[2] = na(bunch_intensity_p2_mod)

            bl_interp_m_coll[1] = na(bl_4sigma_s1_mod)/4.*clight
            bl_interp_m_coll[2] = na(bl_4sigma_s2_mod)/4.*clight

            dexNoise_h = {1:[], 2:[]}
            dexNoise_v = {1:[], 2:[]}
            dexNoise_h[1] = na(dex_noise1)
            dexNoise_v[1] = na(dey_noise1)    
            dexNoise_h[2] = na(dex_noise2)
            dexNoise_v[2] = na(dey_noise2) 
    
            dBPM_h = {1:[], 2:[]}
            dBPM_v = {1:[], 2:[]}
            dBPM_h[1] = na(deltaBPMnow_hb1)
            dBPM_v[1] = na(deltaBPMnow_vb1)    
            dBPM_h[2] = na(deltaBPMnow_hb2)
            dBPM_v[2] = na(deltaBPMnow_vb2)   
    
            xi_h = {1:[], 2:[]}
            xi_v = {1:[], 2:[]}
            xi_h[1] = na(xi_hb1)
            xi_v[1] = na(xi_vb1)    
            xi_h[2] = na(xi_hb2)
            xi_v[2] = na(xi_vb2)  

            dboff_h = {1:[], 2:[]}
            dboff_v = {1:[], 2:[]}
            dboff_h[1] = na(de_boff1h)
            dboff_v[1] = na(de_boff1v)    
            dboff_h[2] = na(de_boff2h)
            dboff_v[2] = na(de_boff2v)  
    
            #handle non-colliding
            family = 'noncoll'
            eh_interp_noncoll = {1:[], 2:[]}
            ev_interp_noncoll = {1:[], 2:[]}
            eh_interp_noncoll_IBScorr = {1:[], 2:[]}
            ev_interp_noncoll_IBScorr = {1:[], 2:[]}
            eh_interp_raw_noncoll = {1:[], 2:[]}
            ev_interp_raw_noncoll = {1:[], 2:[]}
            b_inten_interp_noncoll = {1:[], 2:[]}
            bl_interp_m_noncoll = {1:[], 2:[]}
    
            emith_b1_init =  dict_fill_SB['eh_interp_%s'%family][1]*cor_fact_1h
            emith_b2_init =  dict_fill_SB['eh_interp_%s'%family][2]*cor_fact_2h

            emitv_b1_init =  dict_fill_SB['ev_interp_%s'%family][1]*cor_fact_1v
            emitv_b2_init =  dict_fill_SB['ev_interp_%s'%family][2]*cor_fact_2v
            #print emith_b1_init

            blen_b1_init =  dict_fill_SB['bl_interp_m_%s'%family][1]
            blen_b2_init =  dict_fill_SB['bl_interp_m_%s'%family][2]
    
            for beam_n in [1, 2]:
                #print trange[1]-trange[0]
                (tt_s, _, _, bunch_intensity_temp1, bunch_intensity_temp2,
		    ex_norm_m1_temp, ex_norm_m2_temp, ey_norm_m1_temp, ey_norm_m2_temp, bl_4sigma_s1_temp,
		    bl_4sigma_s2_temp, ex_norm_m1_temp_IBScorr, ex_norm_m2_temp_IBScorr, ey_norm_m1_temp_IBScorr, ey_norm_m2_temp_IBScorr,dex_noise1,dey_noise1,dex_noise2,dey_noise2, deltaBPMnow_hb1,deltaBPMnow_vb1,deltaBPMnow_hb2,deltaBPMnow_vb2, _,_,_,_, _,_,_,_) = \
                LumiModel(gamma=gamma, betastar_m=betastar_m, phi_full_rad_ATLAS=phi_full_rad_ATLAS,
		    phi_full_rad_CMS=phi_full_rad_CMS,
		    bunch_intensityin_p1=dict_fill_SB['b_inten_interp_%s'%family][1], bunch_intensityin_p2=dict_fill_SB['b_inten_interp_%s'%family][2],
		    exin_norm_m1=emith_b1_init*1.0e-6, exin_norm_m2=emith_b2_init*1.0e-6,
		    eyin_norm_m1=emitv_b1_init*1.0e-6, eyin_norm_m2=emitv_b2_init*1.0e-6,
		    blin_4sigma_s1=blen_b1_init*4.0/clight,
		    blin_4sigma_s2=blen_b2_init*4.0/clight,
		    tFill_s=tFill_s,tauSRxy_s=tauSRxy_s, tauSRl_s=tauSRl_s, sigmaBOff_m2=sigmaBOff_m2, sigmaElastic_m2=0., VRF_V=VRF_V,
		    IBSON=1, emitBU=this_model['emitBU'], BoffON=0, coupling=coupling, noiseGrowth=noiseGrowth, boffGrowth=boffGrowth, blengthBU=this_model['blengthBU'], nIPs = 2., dt_s=trange[1]-trange[0],
		    tau_empirical_h1=this_model['tau_empirical_h1_noncoll'],
		    tau_empirical_v1=this_model['tau_empirical_v1_noncoll'],
		    tau_empirical_h2=this_model['tau_empirical_h2_noncoll'],
		    tau_empirical_v2=this_model['tau_empirical_v2_noncoll'], leveling=False)

                eh_interp_noncoll[1] = na(ex_norm_m1_temp)*1.0e6
                ev_interp_noncoll[1] = na(ey_norm_m1_temp)*1.0e6
                eh_interp_noncoll[2] = na(ex_norm_m2_temp)*1.0e6
                ev_interp_noncoll[2] = na(ey_norm_m2_temp)*1.0e6
        
                eh_interp_noncoll_IBScorr[1] = na(ex_norm_m1_temp_IBScorr)*1.0e6
                ev_interp_noncoll_IBScorr[1] = na(ey_norm_m1_temp_IBScorr)*1.0e6
                eh_interp_noncoll_IBScorr[2] = na(ex_norm_m2_temp_IBScorr)*1.0e6
                ev_interp_noncoll_IBScorr[2] = na(ey_norm_m2_temp_IBScorr)*1.0e6
        
                b_inten_interp_noncoll[1] = na(bunch_intensity_temp1)
                b_inten_interp_noncoll[2] = na(bunch_intensity_temp2)
        
                bl_interp_m_noncoll[1] = na(bl_4sigma_s1_temp)/4.*clight
                bl_interp_m_noncoll[2] = na(bl_4sigma_s2_temp)/4.*clight
        
                coupling=na(coupling)
                noiseGrowth=na(noiseGrowth)
                boffGrowth=na(boffGrowth)        

                dexNoise_noncoll_h = {1:[], 2:[]}
                dexNoise_noncoll_v = {1:[], 2:[]}
                dexNoise_noncoll_h[1] = na(dex_noise1)
                dexNoise_noncoll_v[1] = na(dey_noise1)    
                dexNoise_noncoll_h[2] = na(dex_noise2)
                dexNoise_noncoll_v[2] = na(dey_noise2) 
    
                dBPM_noncoll_h = {1:[], 2:[]}
                dBPM_noncoll_v = {1:[], 2:[]}
                dBPM_noncoll_h[1] = na(deltaBPMnow_hb1)
                dBPM_noncoll_v[1] = na(deltaBPMnow_vb1)    
                dBPM_noncoll_h[2] = na(deltaBPMnow_hb2)
                dBPM_noncoll_v[2] = na(deltaBPMnow_vb2)   
        
            dict_save = {\
	'settings':this_model,
	'eh_interp_coll':eh_interp_coll,
	'ev_interp_coll':ev_interp_coll,
	'eh_interp_coll_IBScorr':eh_interp_coll_IBScorr,
	'ev_interp_coll_IBScorr':ev_interp_coll_IBScorr,
	'b_inten_interp_coll':b_inten_interp_coll,
	'bl_interp_m_coll':bl_interp_m_coll,
	'bunch_lumi':{'ATLAS':Luminosity_invm2s_model_ATLAS, 'CMS':Luminosity_invm2s_model_CMS},
	'noiseParams':{'dexNoise_h':dexNoise_h,'dexNoise_v':dexNoise_v,'dBPM_h':dBPM_h,'dBPM_v':dBPM_v,'xi_h':xi_h,'xi_v':xi_v},
	'boffParams':{'dboff_h':dboff_h,'dboff_v':dboff_v},       
	'eh_interp_noncoll':eh_interp_noncoll,
	'ev_interp_noncoll':ev_interp_noncoll,
	'eh_interp_noncoll_IBScorr':eh_interp_noncoll_IBScorr,
	'ev_interp_noncoll_IBScorr':ev_interp_noncoll_IBScorr,
	'b_inten_interp_noncoll':b_inten_interp_noncoll,
	'bl_interp_m_noncoll':bl_interp_m_noncoll,
	'noiseParams_noncoll':{'dexNoise_noncoll_h':dexNoise_noncoll_h,'dexNoise_noncoll_v':dexNoise_noncoll_v,'dBPM_noncoll_h':dBPM_noncoll_h,'dBPM_noncoll_v':dBPM_noncoll_v},                 
	'case': dict_case,
	'time_range':trange}

            dict_models[model_name] = dict_save
            #print 'Fill %d. Start saving pickle...'%filln
            #with gzip.open('../SB_analysis/fill_%d/fill_%d_sbmodel_case%d.pkl.gz'%(filln, filln, case), 'wb') as fid:
            #with gzip.open('/afs/cern.ch/user/l/lumimod/lumimod/2018/procdata/fill_%d/fill_%d_sbmodel_case%d.pkl.gz'%(filln, filln, case), 'wb') as fid:
            if coupling==0 and noiseGrowth==0 and boffGrowth==0:
                with gzip.open('/eos/user/s/stpapado/2018/runModel_forFill/saved_pkl/%d/fill_%d_sbmodel_case%d_oldModel.pkl.gz'%(theyear,filln, case), 'wb') as fid: #for noise dbpm k=1/2
                    pickle.dump(dict_models, fid)
                    print 'Done saving "old model" pickle for fill%d'%filln
            if coupling==1 and noiseGrowth==0 and boffGrowth==0:
                with gzip.open('/eos/user/s/stpapado/2018/runModel_forFill/saved_pkl/%d/fill_%d_sbmodel_case%d_Coupl.pkl.gz'%(theyear,filln, case), 'wb') as fid: #for noise dbpm k=1/2
                    pickle.dump(dict_models, fid)
                    print 'Done saving "coupling" pickle for fill%d'%filln
            if coupling==1 and noiseGrowth==1 and boffGrowth==0:        
                with gzip.open('/eos/user/s/stpapado/2018/runModel_forFill/saved_pkl/%d/fill_%d_sbmodel_case%d_CouplNoise.pkl.gz'%(theyear,filln, case), 'wb') as fid:  #for noise dbpm k=1/2         
                    pickle.dump(dict_models, fid)         
                    print 'Done saving "coupling&noise" pickle for fill%d'%filln
            if coupling==1 and noiseGrowth==0 and boffGrowth==1:        
                with gzip.open('/eos/user/s/stpapado/2018/runModel_forFill/saved_pkl/%d/fill_%d_sbmodel_case%d_CouplBoffcor2.pkl.gz'%(theyear,filln, case), 'wb') as fid:  #for noise dbpm k=1/2         
                    pickle.dump(dict_models, fid)         
                    print 'Done saving "coupling&boff" pickle for fill%d'%filln            
            if coupling and noiseGrowth and boffGrowth:
                with gzip.open('/eos/user/s/stpapado/2018/runModel_forFill/saved_pkl/%d/fill_%d_sbmodel_case%d_CouplNoiseBoffcor2.pkl.gz'%(theyear,filln, case), 'wb') as fid: #for noise dbpm k=1/2
                    pickle.dump(dict_models, fid)
                    print 'Done saving "coupling&noise&boff" pickle for fill%d'%filln

fill6238, 2017 8b4e
xingAngle: [0.0003 0.0003 0.0003 0.0003 0.0003 0.0003 0.0003 0.0003 0.0003 0.0003
 0.0003 0.0003 0.0003 0.0003 0.0003 0.0003 0.0003 0.0003 0.0003 0.0003
 0.0003 0.0003 0.0003 0.0003 0.0003 0.0003 0.0003 0.0003 0.0003 0.0003
 0.0003 0.0003 0.0003 0.0003 0.0003 0.0003 0.0003 0.0003 0.0003 0.0003
 0.0003 0.0003 0.0003 0.0003 0.0003 0.0003 0.0003 0.0003 0.0003 0.0003
 0.0003 0.0003 0.0003 0.0003 0.0003 0.0003 0.0003 0.0003 0.0003 0.0003
 0.0003 0.0003 0.0003 0.0003 0.0003 0.0003 0.0003 0.0003]
beta*: 0.3
Running  "old model" for fill6238
1


/usr/local/bin/start_ipykernel.py:11: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/bin/start_ipykernel.py:12: IntegrationWarning: The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 
  underestimated.


2
3
4


KeyboardInterrupt: 